# Best Neighbourhood in Toronto for new restaurant

### Problem Statement: 
John Smith is a good chief. After discussion with his family, he decides to open a new restaurant in Toronto. However, he doesn't know which is the best neighbourhood in Toronto for a new restaurant. He asks us to research and help him figure out the best neighbourhood. 

### Data Source: 
we will use the toronto data generated from last assignment, which contains the Borough, Postal Code, Neighbourhood, Latitude, Longitude. Further, we will use Foursuare API to pull venue data to find out the neighbourhood with most venues but least restaurants.

# Methodology

In [1]:
# Import and load the toronto data from the previous assignment's result

from project_lib import Project
project = Project(project_id="f3101ff4-e278-4a2c-8141-00afc30b3e09",project_access_token="p-d28f16c105a5ce75a81225274d24530c67c9bc51")

# Fetch the file
my_file = project.get_file("torontodata.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
import pandas as pd
torontodata = pd.read_csv(my_file)

In [2]:
# Explore the data
torontodata.head()

,Cluster,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Borough_num
0,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1
1,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1
2,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1
3,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1
4,3,M4E,East Toronto,The Beaches,43.676357,-79.293031,2


In [29]:
# Import necessary libraries

import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print('Libraries imported.')

Folium installed
Libraries imported.
Your credentails:
CLIENT_ID: MJC4YORCIR53PFIRETELJTH5U54AGWB4ZOVQSPSXQB0ULUMM
CLIENT_SECRET:SRNCARHPWTD4GKHNRS02VIQ0ZVENWGXF2H0PHG5JTQY1N1TM


In [34]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: MJC4YORCIR53PFIRETELJTH5U54AGWB4ZOVQSPSXQB0ULUMM
CLIENT_SECRET:SRNCARHPWTD4GKHNRS02VIQ0ZVENWGXF2H0PHG5JTQY1N1TM


In [35]:
# Define a function to get all venues near a neighborhood in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# run the function on each neighborhood and generate dataframe 
toronto_venues = getNearbyVenues(torontodata.Neighbourhood, torontodata.Latitude, torontodata.Longitude, radius=500)
toronto_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


# Results

In [73]:
# Generate neighbourhoods with the number of total venues within a radius of 500m.
totalvenue = toronto_venues.groupby('Neighbourhood').count().sort_values(by='Venue', ascending=False)[['Venue']]
totalvenue.rename(columns={'Venue':'Venue Number'}, inplace=True)
totalvenue.head()

,Venue Number
Neighbourhood,
"Toronto Dominion Centre, Design Exchange",100
"Commerce Court, Victoria Hotel",100
"First Canadian Place, Underground city",100
"Garden District, Ryerson",100
"Harbourfront East, Union Station, Toronto Islands",100


In [74]:
# Generate neighbourhoods with the number of total restaurants within a radius of 500m.
restaurantvenue = toronto_venues[toronto_venues['Venue Category'] == 'Restaurant']
topneigh = restaurantvenue.groupby('Neighbourhood').count().sort_values(by='Venue', ascending=False)[['Venue']]
topneigh.rename(columns={'Venue':'Restaurant Number'}, inplace=True)
topneigh.head()

,Restaurant Number
Neighbourhood,
"Commerce Court, Victoria Hotel",7
"Toronto Dominion Centre, Design Exchange",4
"First Canadian Place, Underground city",4
"Richmond, Adelaide, King",4
"Harbourfront East, Union Station, Toronto Islands",3


In [75]:
# merge the number of venues and restaurants into one dataframe
bestneigh = pd.merge(totalvenue, topneigh, on='Neighbourhood')
topneigh = bestneigh[bestneigh['Venue Number'] == bestneigh['Venue Number'].max()].sort_values('Restaurant Number',ascending=True)
topneigh

,Venue Number,Restaurant Number
Neighbourhood,,
"Garden District, Ryerson",100,1
"Harbourfront East, Union Station, Toronto Islands",100,3
"Toronto Dominion Centre, Design Exchange",100,4
"First Canadian Place, Underground city",100,4
"Commerce Court, Victoria Hotel",100,7


# Discussion

### limitation
The Foursquare explore API has a limit of 100 venues, which can't effectively identify the real total number of venues near the neighbourhood

# Conclusion

In [76]:
print('The Neighbourhood {} is the best one because it has the most venues and least restaurants'.format(str(topneigh.index[0])))

The Neighbourhood Garden District, Ryerson is the best one because it has the most venues and least restaurants
